In [1]:
import second.builder.anchor_generator_builder as ab


In [63]:
config_path="/home/gx/GitHub/second.pytorch/second/configs/car.fhd.config"
from second.protos import pipeline_pb2
from google.protobuf import text_format
config = pipeline_pb2.TrainEvalPipelineConfig()
with open(config_path, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, config)

In [64]:
from second.pytorch.train import build_network
input_cfg = config.train_input_reader
eval_input_cfg = config.eval_input_reader
model_cfg = config.model.second
train_cfg = config.train_config
net = build_network(model_cfg, False)

In [65]:
model_cfg

network_class_name: "VoxelNet"
voxel_generator {
  voxel_size: 0.05000000074505806
  voxel_size: 0.05000000074505806
  voxel_size: 0.10000000149011612
  point_cloud_range: 0.0
  point_cloud_range: -40.0
  point_cloud_range: -3.0
  point_cloud_range: 70.4000015258789
  point_cloud_range: 40.0
  point_cloud_range: 1.0
  max_number_of_points_per_voxel: 5
}
voxel_feature_extractor {
  module_class_name: "SimpleVoxel"
  num_filters: 16
  num_input_features: 4
}
middle_feature_extractor {
  module_class_name: "SpMiddleFHD"
  num_input_features: 4
  downsample_factor: 8
}
rpn {
  module_class_name: "RPNV2"
  layer_nums: 5
  layer_strides: 1
  num_filters: 128
  upsample_strides: 1.0
  num_upsample_filters: 128
  num_groups: 32
  num_input_features: 128
}
num_point_features: 4
use_sigmoid_score: true
loss {
  localization_loss {
    weighted_smooth_l1 {
      sigma: 3.0
      code_weight: 1.0
      code_weight: 1.0
      code_weight: 1.0
      code_weight: 1.0
      code_weight: 1.0
      code

In [66]:
import second.builder.target_assigner_builder as tab

In [67]:
target_cfg = config.model.second.target_assigner

In [68]:
ta = tab.build(target_cfg, )

TypeError: build() missing 2 required positional arguments: 'bv_range' and 'box_coder'

In [69]:
from second.builder import target_assigner_builder, voxel_builder
from second.pytorch.builder import (box_coder_builder, input_reader_builder,
                                    lr_scheduler_builder, optimizer_builder,
                                    second_builder)

voxel_generator = voxel_builder.build(model_cfg.voxel_generator)
bv_range = voxel_generator.point_cloud_range[[0, 1, 3, 4]]
box_coder = box_coder_builder.build(model_cfg.box_coder)
target_assigner_cfg = model_cfg.target_assigner
target_assigner = target_assigner_builder.build(target_assigner_cfg,
                                                    bv_range, box_coder)
box_coder.custom_ndim = target_assigner._anchor_generators[0].custom_ndim

In [70]:
bv_range

array([  0. , -40. ,  70.4,  40. ], dtype=float32)

In [71]:
box_coder

In [72]:
box_coder.code_size

7

In [73]:
box_coder.custom_ndim

0

In [74]:
box_coder.linear_dim

False

In [75]:
# box encode: 
# input box: [N, 7+?] Tensor, xyzwlhr, custom values
# anchors  [N, 7] Tensor, xyzwlhr
# return encoded xyzwlhr

In [76]:
model_cfg.box_coder

ground_box3d_coder {
}

In [77]:
target_assigner_cfg

class_settings {
  anchor_generator_range {
    sizes: 1.600000023841858
    sizes: 3.9000000953674316
    sizes: 1.559999942779541
    anchor_ranges: 0.0
    anchor_ranges: -40.0
    anchor_ranges: -1.0
    anchor_ranges: 70.4000015258789
    anchor_ranges: 40.0
    anchor_ranges: -1.0
    rotations: 0.0
    rotations: 1.5700000524520874
  }
  region_similarity_calculator {
    nearest_iou_similarity {
    }
  }
  use_rotate_nms: true
  nms_pre_max_size: 1000
  nms_post_max_size: 100
  nms_score_threshold: 0.30000001192092896
  nms_iou_threshold: 0.009999999776482582
  matched_threshold: 0.6000000238418579
  unmatched_threshold: 0.44999998807907104
  class_name: "Car"
}
sample_positive_fraction: -1.0
sample_size: 512
assign_per_class: true

In [78]:
target_assigner.__dir__()

['_box_coder',
 '_anchor_generators',
 '_sim_calcs',
 '_positive_fraction',
 '_sample_size',
 '_classes',
 '_assign_per_class',
 '_feature_map_sizes',
 '__module__',
 '__init__',
 'box_coder',
 'classes',
 'assign',
 'assign_all',
 'assign_per_class',
 'generate_anchors',
 'generate_anchors_dict',
 'num_anchors_per_location',
 'box_ndim',
 'num_anchors',
 'anchors_range',
 'num_anchors_per_location_class',
 '__dict__',
 '__weakref__',
 '__doc__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [79]:
grid_size = voxel_generator.grid_size
grid_size

array([1408, 1600,   40])

In [80]:
from second.utils.config_tool import get_downsample_factor

out_size_factor = get_downsample_factor(model_cfg)
out_size_factor

8

In [81]:
feature_map_size = grid_size[:2] // out_size_factor
feature_map_size

array([176, 200])

In [82]:
feature_map_size = [*feature_map_size, 1][::-1]
feature_map_size  # [1,64,64]

[1, 200, 176]

In [83]:
anchors = target_assigner.generate_anchors(feature_map_size)

feautres size [1, 200, 176]
res = (1, 200, 176, 1, 2, 7)


In [84]:
anchors

{'anchors': array([[  0.        , -40.        ,  -1.        , ...,   3.9       ,
           1.56      ,   0.        ],
        [  0.40228572, -40.        ,  -1.        , ...,   3.9       ,
           1.56      ,   0.        ],
        [  0.80457145, -40.        ,  -1.        , ...,   3.9       ,
           1.56      ,   0.        ],
        ...,
        [ 69.59543   ,  40.        ,  -1.        , ...,   3.9       ,
           1.56      ,   1.57      ],
        [ 69.99772   ,  40.        ,  -1.        , ...,   3.9       ,
           1.56      ,   1.57      ],
        [ 70.4       ,  40.        ,  -1.        , ...,   3.9       ,
           1.56      ,   1.57      ]], dtype=float32),
 'matched_thresholds': array([0.6, 0.6, 0.6, ..., 0.6, 0.6, 0.6], dtype=float32),
 'unmatched_thresholds': array([0.45, 0.45, 0.45, ..., 0.45, 0.45, 0.45], dtype=float32)}

In [85]:
anchors['anchors'].shape

(70400, 7)

In [86]:
# generate 64 * 64 *2  anchors
anchors['anchors'][0]

array([  0.  , -40.  ,  -1.  ,   1.6 ,   3.9 ,   1.56,   0.  ],
      dtype=float32)

In [87]:
for i in range(100):
    print(anchors['anchors'][i])

[  0.   -40.    -1.     1.6    3.9    1.56   0.  ]
[  0.40228572 -40.          -1.           1.6          3.9
   1.56         0.        ]
[  0.80457145 -40.          -1.           1.6          3.9
   1.56         0.        ]
[  1.2068572 -40.         -1.          1.6         3.9         1.56
   0.       ]
[  1.6091429 -40.         -1.          1.6         3.9         1.56
   0.       ]
[  2.0114286 -40.         -1.          1.6         3.9         1.56
   0.       ]
[  2.4137144 -40.         -1.          1.6         3.9         1.56
   0.       ]
[  2.816 -40.     -1.      1.6     3.9     1.56    0.   ]
[  3.2182858 -40.         -1.          1.6         3.9         1.56
   0.       ]
[  3.6205716 -40.         -1.          1.6         3.9         1.56
   0.       ]
[  4.022857 -40.        -1.         1.6        3.9        1.56
   0.      ]
[  4.425143 -40.        -1.         1.6        3.9        1.56
   0.      ]
[  4.827429 -40.        -1.         1.6        3.9        1.56
   0.     

In [88]:
net.rpn

RPNV2(
  (blocks): ModuleList(
    (0): Sequential(
      (0): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (2): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (3): ReLU()
      (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (5): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (8): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (9): ReLU()
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (11): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (12): ReLU()
      (13): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (